# Zero-Touch Environment Verification

This notebook verifies that our Codespaces environment provides a true zero-touch experience for students:
- ✅ Jupyter Lab accessible without authentication
- ✅ Python data science stack ready
- ✅ R kernel available
- ✅ PostgreSQL accessible without passwords
- ✅ All running as jovyan user (official Jupyter Docker Stacks default)

**Students should be able to run all cells without any password prompts or configuration!**

## 1. Environment Check

In [ ]:
import os
import getpass
import platform

print("🔍 Environment Information:")
print(f"   User: {getpass.getuser()}")
print(f"   Home: {os.path.expanduser('~')}")
print(f"   Platform: {platform.platform()}")
print(f"   Python: {platform.python_version()}")
print(f"   Working Directory: {os.getcwd()}")

## 2. Python Data Science Stack Test

In [ ]:
# Test core data science packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import sklearn
import scipy

print("📊 Data Science Packages:")
print(f"   NumPy: {np.__version__}")
print(f"   Pandas: {pd.__version__}")
print(f"   Matplotlib: {plt.matplotlib.__version__}")
print(f"   Seaborn: {sns.__version__}")
print(f"   Plotly: {px.__version__}")
print(f"   Scikit-learn: {sklearn.__version__}")
print(f"   SciPy: {scipy.__version__}")

# Quick data manipulation test
df = pd.DataFrame({
    'x': np.random.randn(100),
    'y': np.random.randn(100)
})
print(f"\n✅ Created test DataFrame with {len(df)} rows")
print(df.head())

## 3. PostgreSQL Connection Test (Zero Password)

In [ ]:
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine, text

try:
    # Test direct psycopg2 connection (should work with trust auth)
    conn = psycopg2.connect(
        host="localhost",
        database="jovyan",
        user="jovyan"
        # Note: NO PASSWORD - this should work with trust authentication
    )
    
    cursor = conn.cursor()
    cursor.execute("SELECT current_user, current_database(), version();")
    result = cursor.fetchone()
    
    print("🗄️ PostgreSQL Connection (psycopg2):")
    print(f"   User: {result[0]}")
    print(f"   Database: {result[1]}")
    print(f"   Version: {result[2][:50]}...")
    
    cursor.close()
    conn.close()
    print("✅ Direct PostgreSQL connection successful (no password needed)")
    
except Exception as e:
    print(f"❌ PostgreSQL connection failed: {e}")

In [ ]:
# Test SQLAlchemy connection
try:
    engine = create_engine('postgresql://jovyan@localhost/jovyan')
    
    with engine.connect() as conn:
        result = conn.execute(text("SELECT 'SQLAlchemy connection works!' as message"))
        message = result.fetchone()[0]
        print(f"🔗 {message}")
        
        # Create a test table
        conn.execute(text("""
            CREATE TABLE IF NOT EXISTS test_table (
                id SERIAL PRIMARY KEY,
                name VARCHAR(50),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """))
        
        # Insert test data
        conn.execute(text("""
            INSERT INTO test_table (name) VALUES 
            ('Student Test 1'), 
            ('Student Test 2')
            ON CONFLICT DO NOTHING
        """))
        
        conn.commit()
        print("✅ SQLAlchemy connection and table operations successful")
        
except Exception as e:
    print(f"❌ SQLAlchemy connection failed: {e}")

## 4. Pandas + PostgreSQL Integration Test

In [ ]:
# Test pandas read_sql with PostgreSQL
try:
    engine = create_engine('postgresql://jovyan@localhost/jovyan')
    
    # Read data into pandas
    df_from_db = pd.read_sql(
        "SELECT * FROM test_table ORDER BY id", 
        engine
    )
    
    print("📋 Data from PostgreSQL:")
    print(df_from_db)
    
    # Create sample data and write to database
    sample_data = pd.DataFrame({
        'product': ['Widget A', 'Widget B', 'Widget C'],
        'price': [10.99, 15.49, 8.75],
        'quantity': [100, 50, 200]
    })
    
    sample_data.to_sql('products', engine, if_exists='replace', index=False)
    print("\n✅ Successfully wrote pandas DataFrame to PostgreSQL")
    
    # Read it back
    products_df = pd.read_sql("SELECT * FROM products", engine)
    print("\n📦 Products table:")
    print(products_df)
    
except Exception as e:
    print(f"❌ Pandas-PostgreSQL integration failed: {e}")

## 5. Data Visualization Test

In [ ]:
# Test matplotlib
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
x = np.linspace(0, 10, 100)
plt.plot(x, np.sin(x), label='sin(x)')
plt.plot(x, np.cos(x), label='cos(x)')
plt.title('Matplotlib Test')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
data = np.random.randn(1000)
plt.hist(data, bins=30, alpha=0.7)
plt.title('Random Data Distribution')
plt.xlabel('Value')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

print("✅ Matplotlib visualization successful")

In [ ]:
# Test plotly interactive visualization
try:
    from IPython.display import display
    
    # Create sample data
    sample_df = pd.DataFrame({
        'x': np.random.randn(100),
        'y': np.random.randn(100),
        'category': np.random.choice(['A', 'B', 'C'], 100)
    })
    
    fig = px.scatter(
        sample_df, 
        x='x', 
        y='y', 
        color='category',
        title='Interactive Plotly Scatter Plot'
    )
    
    fig.show()
    print("✅ Plotly interactive visualization successful")
    
except Exception as e:
    print(f"⚠️ Plotly visualization issue: {e}")

## 6. File System and Permissions Test

In [ ]:
import stat

# Test file operations
test_file = '/tmp/student_test.txt'

try:
    # Write test file
    with open(test_file, 'w') as f:
        f.write("Hello from the data science classroom!\n")
        f.write(f"Written by user: {getpass.getuser()}\n")
    
    # Read it back
    with open(test_file, 'r') as f:
        content = f.read()
    
    print("📁 File Operations Test:")
    print(f"   File created: {test_file}")
    print(f"   Content: {content.strip()}")
    
    # Check permissions
    file_stat = os.stat(test_file)
    permissions = stat.filemode(file_stat.st_mode)
    print(f"   Permissions: {permissions}")
    
    # Clean up
    os.remove(test_file)
    print("✅ File operations successful")
    
except Exception as e:
    print(f"❌ File operations failed: {e}")

## 7. Jupyter Configuration Check

In [ ]:
# Check Jupyter configuration
jupyter_config_path = os.path.expanduser('~/.jupyter/jupyter_server_config.py')

print("🔧 Jupyter Configuration:")
if os.path.exists(jupyter_config_path):
    with open(jupyter_config_path, 'r') as f:
        config_content = f.read()
    print(f"   Config file exists: {jupyter_config_path}")
    print("   Content:")
    for line in config_content.split('\n'):
        if line.strip():
            print(f"     {line}")
    
    # Check for zero-touch settings
    if "c.ServerApp.token = ''" in config_content:
        print("   ✅ Token authentication disabled")
    if "c.ServerApp.password = ''" in config_content:
        print("   ✅ Password authentication disabled")
        
else:
    print(f"   ⚠️ Config file not found: {jupyter_config_path}")

# Check environment variables
print("\n🌍 PostgreSQL Environment Variables:")
pg_vars = ['PGUSER', 'PGDATABASE', 'PGHOST', 'PGPORT']
for var in pg_vars:
    value = os.environ.get(var, 'Not set')
    print(f"   {var}: {value}")

## 8. Summary Report

In [ ]:
print("🎓 ZERO-TOUCH ENVIRONMENT VERIFICATION COMPLETE")
print("=" * 50)
print("")
print("If all tests above passed, students should experience:")
print("")
print("✅ Immediate Jupyter Lab access (no authentication)")
print("✅ Full Python data science stack ready")
print("✅ PostgreSQL database accessible (no passwords)")
print("✅ Seamless pandas ↔ PostgreSQL integration")
print("✅ Interactive visualizations working")
print("✅ All file operations permitted")
print("✅ Running as jovyan user (official Jupyter standard)")
print("")
print("🚀 Environment is ready for data science education!")
print("")
print("Students can now:")
print("  • Open Codespace and start coding immediately")
print("  • Use Jupyter notebooks without any setup")
print("  • Connect to databases without passwords")
print("  • Focus on learning, not configuration")